# 病床姿勢監測 - 資料準備與對齊（Version 4 - 修正累積誤差）

**專案**: 熱成像超解析度 + 姿勢辨識  
**作者**: rochi190  
**日期**: 2025-11-06  
**版本**: v4 - 修正 Thermal 時間戳累積誤差問題

## 修正重點
- ✅ **使用全域幀計數器**，避免秒級分組的累積誤差
- ✅ **微秒計算修正**：直接計算微秒而非毫秒*1000
- ✅ **時間戳不再被修改**：保持原始配對時間的一致性
- ✅ **簡化配對邏輯**：直接按時間戳配對，移除複雜的 burst/dropout 處理

## 功能
1. 讀取 output.txt 取得相機開始時間
2. 載入熱成像資料（全域幀計數器，無累積誤差）
3. 分析 RGB 影片資訊
4. **直接時間戳配對**（簡單且準確）
5. 翻轉檢測與修復
6. 匯出 RGB 圖片供 LabelMe 標註
7. 儲存對齊後的 Thermal 資料

## 輸出
- `output/labelme_project/` - LabelMe 標註專案
- `output/aligned_dataset/` - 對齊後的訓練資料
- `output/pairing_analysis/` - 配對分析結果

---
## Step 0: 環境設定與套件匯入

In [115]:
import os
import sys
import json
import re
import cv2
import numpy as np
import multiprocessing as mp
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # 使用非互動式後端（適用於 SSH 環境）
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional, Any
from tqdm import tqdm
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings('ignore')

# Matplotlib 設定
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.dpi'] = 100

# Seaborn 樣式
sns.set_style('whitegrid')

print("="*70)
print("病床姿勢監測系統 - 資料準備模組 v4（修正累積誤差）")
print("="*70)
print(f"執行時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"使用者: rochi190")
print(f"Python 版本: {sys.version}")
print(f"NumPy 版本: {np.__version__}")
print(f"OpenCV 版本: {cv2.__version__}")
print("="*70)

病床姿勢監測系統 - 資料準備模組 v4（修正累積誤差）
執行時間: 2025-11-09 00:17:29
使用者: rochi190
Python 版本: 3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]
NumPy 版本: 2.2.6
OpenCV 版本: 4.10.0


---
## Step 1: 路徑與參數配置

In [116]:
# ========================================
# 路徑配置
# ========================================
NUM_WORKERS = max(1, mp.cpu_count() - 1)
BASE_DIR = Path(os.getcwd())
DATA_DIR = BASE_DIR / '01'
THERMAL_DIR = DATA_DIR / 'thermal'
CAMERA_DIR = DATA_DIR / 'camera'
OUTPUT_TXT = CAMERA_DIR / 'output.txt'
AVI_FILE = CAMERA_DIR / 'output.avi'

# 輸出目錄
OUTPUT_DIR = BASE_DIR / 'output_v6'
PAIRING_DIR = OUTPUT_DIR / 'pairing_analysis'
LABELME_DIR = OUTPUT_DIR / 'labelme_project'
ALIGNED_DIR = OUTPUT_DIR / 'aligned_dataset'
DIAGNOSTIC_DIR = OUTPUT_DIR / 'diagnostics'

for directory in [PAIRING_DIR, LABELME_DIR, ALIGNED_DIR, DIAGNOSTIC_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# ========================================
# 熱成像參數
# ========================================
THERMAL_RESOLUTION = (32, 24)  # (寬度, 高度)
THERMAL_WIDTH, THERMAL_HEIGHT = THERMAL_RESOLUTION

# ========================================
# 配對參數
# ========================================
TARGET_FPS = 8  # 目標 FPS
FRAME_INTERVAL_MS = 1000 / TARGET_FPS  # 125ms

# 翻轉設定（執行 Step 7 後調整）
FLIP_THERMAL = True
FLIP_RGB = False

# ========================================
# 顯示配置
# ========================================
print(f"\n📁 路徑檢查:")
print(f"  Thermal 目錄: {'✓' if THERMAL_DIR.exists() else '✗'} {THERMAL_DIR}")
print(f"  Camera 目錄: {'✓' if CAMERA_DIR.exists() else '✗'} {CAMERA_DIR}")
print(f"  output.txt: {'✓' if OUTPUT_TXT.exists() else '✗'} {OUTPUT_TXT}")
print(f"  AVI 檔案: {'✓' if AVI_FILE.exists() else '✗'} {AVI_FILE}")

print(f"\n⚙️ 配對參數:")
print(f"  目標 FPS: {TARGET_FPS}")
print(f"  幀間隔: {FRAME_INTERVAL_MS:.1f}ms")

print(f"\n🔄 翻轉設定:")
print(f"  翻轉 Thermal: {FLIP_THERMAL}")
print(f"  翻轉 RGB: {FLIP_RGB}")


📁 路徑檢查:
  Thermal 目錄: ✓ /home/kirito/USED/01/thermal
  Camera 目錄: ✓ /home/kirito/USED/01/camera
  output.txt: ✓ /home/kirito/USED/01/camera/output.txt
  AVI 檔案: ✓ /home/kirito/USED/01/camera/output.avi

⚙️ 配對參數:
  目標 FPS: 8
  幀間隔: 125.0ms

🔄 翻轉設定:
  翻轉 Thermal: True
  翻轉 RGB: False


---
## Step 2: 解析 output.txt（相機開始時間）

In [117]:
def parse_output_txt(txt_path: Path) -> Optional[datetime]:
    """
    解析 output.txt 取得相機開始時間
    """
    if not txt_path.exists():
        print(f"❌ {txt_path} 不存在")
        return None
    
    try:
        with open(txt_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except:
        with open(txt_path, 'r', encoding='big5') as f:
            content = f.read()
    
    print(f"\n{'='*70}")
    print(f"解析 output.txt")
    print(f"{'='*70}")
    print(f"檔案大小: {len(content)} bytes")
    print(f"\n完整內容:\n{content}")
    print(f"{'='*70}")
    
    # 時間格式模式
    patterns = [
        (r'(\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2})', '%Y-%m-%d %H:%M:%S'),
        (r'(\d{4}/\d{2}/\d{2}\s+\d{2}:\d{2}:\d{2})', '%Y/%m/%d %H:%M:%S'),
    ]
    
    for pattern, fmt in patterns:
        match = re.search(pattern, content)
        if match:
            try:
                camera_start = datetime.strptime(match.group(1), fmt)
                print(f"\n✅ 找到相機開始時間: {camera_start}")
                return camera_start
            except:
                continue
    
    print(f"\n❌ 無法解析時間")
    return None

# 執行解析
camera_start_time = parse_output_txt(OUTPUT_TXT)

if camera_start_time is None:
    raise RuntimeError("❌ 無法解析 output.txt，請檢查檔案格式")

print(f"\n📌 相機開始時間: {camera_start_time}")


解析 output.txt
檔案大小: 899 bytes

完整內容:
viedo_start_2020-09-24 13:09:29 
lay_mid_2020-09-24 13:11:08 
end_time_2020-09-24 13:11:43 
lay_right_2020-09-24 13:12:02 
end_time_2020-09-24 13:12:39 
lay_left_2020-09-24 13:12:58 
end_time_2020-09-24 13:13:36 
bed_2020-09-24 13:13:53 
end_time_2020-09-24 13:14:32 
sit_right_2020-09-24 13:14:40 
end_time_2020-09-24 13:15:22 
sit_left_2020-09-24 13:15:27 
end_time_2020-09-24 13:16:04 
Falldown_2020-09-24 13:16:38 
end_time_2020-09-24 13:16:43 
Falldown_2020-09-24 13:16:57 
end_time_2020-09-24 13:17:01 
Falldown_2020-09-24 13:17:38 
end_time_2020-09-24 13:17:42 
Falldown_2020-09-24 13:17:53 
end_time_2020-09-24 13:17:58 
Falldown_2020-09-24 13:18:09 
end_time_2020-09-24 13:18:12 
Falldown_2020-09-24 13:18:34 
end_time_2020-09-24 13:18:38 
Falldown_2020-09-24 13:19:06 
end_time_2020-09-24 13:19:08 
Falldown_2020-09-24 13:20:07 
Falldown_2020-09-24 13:20:09 
Falldown_2020-09-24 13:20:18 


✅ 找到相機開始時間: 2020-09-24 13:09:29

📌 相機開始時間: 2020-09-24 13:09:2

---
## Step 3: 載入熱成像資料（全域計數器版本 - 修正累積誤差）

In [118]:
def load_thermal_data_with_global_counter(log_files: List[Path]) -> List[Dict]:
    """
    載入熱成像資料（使用全域幀計數器，避免累積誤差）
    
    修正重點：
    1. 使用全域幀計數器，不依賴秒級分組
    2. 從第一幀開始，每幀間隔固定 125ms
    3. 避免 burst/dropout 導致的時間跳躍
    """
    print(f"\n{'='*70}")
    print(f"載入熱成像資料（全域計數器版本）")
    print(f"{'='*70}")
    print(f"找到 {len(log_files)} 個 log 檔案")
    print(f"🔧 使用全域幀計數器，每幀間隔 {FRAME_INTERVAL_MS:.1f}ms")
    
    all_data = []
    global_frame_count = 0
    first_timestamp = None
    
    for log_file in tqdm(log_files, desc="載入 Thermal"):
        try:
            with open(log_file, 'r', encoding='utf-8') as f:
                for line in f:
                    line = line.strip()
                    if not line:
                        continue
                    
                    try:
                        data_dict = json.loads(line)
                        time_parts = data_dict.get('time', [])
                        
                        if len(time_parts) < 6:
                            continue
                        
                        year, month, day, hour, minute, second = time_parts[:6]
                        original_timestamp = datetime(year, month, day, hour, minute, second)
                        
                        # 記錄第一幀的時間戳作為基準
                        if first_timestamp is None:
                            first_timestamp = original_timestamp
                        
                        # 🔧 使用全域計數器計算精確時間戳
                        offset_ms = global_frame_count * FRAME_INTERVAL_MS
                        timestamp = first_timestamp + timedelta(milliseconds=offset_ms)
                        
                        message = data_dict.get('message', [])
                        if len(message) != THERMAL_WIDTH * THERMAL_HEIGHT:
                            continue
                        
                        thermal_image = np.array(message, dtype=np.uint8).reshape(
                            THERMAL_HEIGHT, THERMAL_WIDTH
                        )
                        
                        all_data.append({
                            'timestamp': timestamp,
                            'image': thermal_image,
                            'frame_number': global_frame_count,
                            'original_timestamp': original_timestamp
                        })
                        
                        global_frame_count += 1
                        
                    except Exception as e:
                        continue
        
        except Exception as e:
            print(f"❌ 載入失敗 {log_file.name}: {e}")
            continue
    
    print(f"\n✅ 載入完成")
    print(f"  總幀數: {len(all_data):,}")
    
    if all_data:
        thermal_start = all_data[0]['timestamp']
        thermal_end = all_data[-1]['timestamp']
        duration = (thermal_end - thermal_start).total_seconds()
        
        print(f"  第一幀: {thermal_start}")
        print(f"  最後幀: {thermal_end}")
        print(f"  時長: {duration:.2f} 秒")
        print(f"  平均 FPS: {len(all_data) / duration:.2f}")
        
        # 驗證時間間隔
        time_diffs = []
        for i in range(min(100, len(all_data) - 1)):
            dt = (all_data[i+1]['timestamp'] - all_data[i]['timestamp']).total_seconds() * 1000
            time_diffs.append(dt)
        
        print(f"\n  時間間隔統計（前 100 幀）:")
        print(f"    平均: {np.mean(time_diffs):.1f}ms")
        print(f"    標準差: {np.std(time_diffs):.1f}ms")
        print(f"    最小值: {min(time_diffs):.1f}ms")
        print(f"    最大值: {max(time_diffs):.1f}ms")
        print(f"    預期: {FRAME_INTERVAL_MS:.1f}ms")
    
    print("="*70)
    
    return all_data

# 載入所有 thermal log
log_files = sorted(THERMAL_DIR.glob('log*.txt'))

if not log_files:
    raise FileNotFoundError(f"❌ 未找到 log 檔案於 {THERMAL_DIR}")

therm_data = load_thermal_data_with_global_counter(log_files)


載入熱成像資料（全域計數器版本）
找到 5 個 log 檔案
🔧 使用全域幀計數器，每幀間隔 125.0ms


載入 Thermal: 100%|██████████| 5/5 [00:00<00:00,  9.65it/s]


✅ 載入完成
  總幀數: 5,318
  第一幀: 2020-09-24 13:09:30
  最後幀: 2020-09-24 13:20:34.625000
  時長: 664.62 秒
  平均 FPS: 8.00

  時間間隔統計（前 100 幀）:
    平均: 125.0ms
    標準差: 0.0ms
    最小值: 125.0ms
    最大值: 125.0ms
    預期: 125.0ms


---
## Step 4: 分析 RGB 影片

In [119]:
# 分析 AVI 影片
cap = cv2.VideoCapture(str(AVI_FILE))

if not cap.isOpened():
    raise RuntimeError(f"❌ 無法開啟影片 {AVI_FILE}")

fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
duration = frame_count / fps if fps > 0 else 0

cap.release()

print(f"\n{'='*70}")
print(f"RGB 影片資訊")
print(f"{'='*70}")
print(f"  FPS: {fps:.2f}")
print(f"  總幀數: {frame_count:,}")
print(f"  解析度: {width} × {height}")
print(f"  時長: {duration:.2f} 秒")
print(f"  開始時間: {camera_start_time}")
print(f"  結束時間: {camera_start_time + timedelta(seconds=duration)}")
print(f"\n  預期 Thermal-RGB 幀間隔:")
print(f"    Thermal 間隔: {FRAME_INTERVAL_MS:.1f}ms")
print(f"    RGB 間隔: {1000/fps:.1f}ms")
print(f"    每個 Thermal 對應 RGB 幀數: {FRAME_INTERVAL_MS / (1000/fps):.1f}")
print("="*70)


RGB 影片資訊
  FPS: 24.67
  總幀數: 16,284
  解析度: 640 × 360
  時長: 660.07 秒
  開始時間: 2020-09-24 13:09:29
  結束時間: 2020-09-24 13:20:29.072963

  預期 Thermal-RGB 幀間隔:
    Thermal 間隔: 125.0ms
    RGB 間隔: 40.5ms
    每個 Thermal 對應 RGB 幀數: 3.1


---
## Step 5: 簡化配對（直接按時間戳配對）

In [120]:
# ========================================
# Step 5: 簡化配對（直接按時間戳配對）
# ========================================
# 改進版本：檢測時間偏移，從較晚的時間開始配對

from collections import defaultdict, Counter

def second_based_pairing_with_time_offset(
    thermal_data: List[Dict],
    camera_start_time: datetime,
    fps: float,
    frame_count: int,
    target_fps: int = 8,
    frame_interval_ms: float = 125.0
) -> Tuple[List[Dict], Dict]:
    """
    秒級分組配對邏輯（改進版本 - 帶時間偏移檢測）
    
    配對流程：
    1. 檢測 RGB 和 Thermal 的起始時間差異
    2. 如果兩者起始時間不同，從較晚的時間開始配對
    3. 按RGB每一秒切割，每0.125秒切一張圖片（每秒8張）
    4. Thermal 按秒分組，每秒最多8張
    5. 檢測缺失的秒和溢出的秒
    6. 用溢出的秒資料補充缺失的秒
    7. 不足8張的秒只取前N張RGB幀配對
    """
    print(f"\n{'='*70}")
    print(f"秒級分組配對（改進版 - 帶時間偏移檢測）")
    print(f"{'='*70}")
    
    FRAMES_PER_SECOND = target_fps  # 8
    FRAME_INTERVAL = frame_interval_ms / 1000  # 0.125 秒
    
    # ========================================
    # 檢測時間偏移
    # ========================================
    print(f"\n📊 檢測時間偏移：")
    
    # RGB 的時間範圍
    rgb_start_time = camera_start_time
    rgb_end_time = camera_start_time + timedelta(seconds=frame_count / fps)
    
    print(f"  RGB 時間範圍: {rgb_start_time} ~ {rgb_end_time}")
    
    # Thermal 的時間範圍
    if thermal_data:
        thermal_start_time = thermal_data[0]['timestamp']
        thermal_end_time = thermal_data[-1]['timestamp']
        print(f"  Thermal 時間範圍: {thermal_start_time} ~ {thermal_end_time}")
    else:
        raise ValueError("❌ Thermal 數據為空")
    
    # 找出較晚的起始時間
    pairing_start_time = max(rgb_start_time, thermal_start_time)
    time_offset = (pairing_start_time - rgb_start_time).total_seconds()
    thermal_offset = (pairing_start_time - thermal_start_time).total_seconds()
    
    print(f"\n🔄 時間對齊:")
    if time_offset > 0:
        print(f"  ⚠️ RGB 比 Thermal 早 {time_offset:.2f} 秒")
        print(f"     將丟掉前 {time_offset:.2f} 秒的 RGB 數據")
    if thermal_offset > 0:
        print(f"  ⚠️ Thermal 比 RGB 早 {thermal_offset:.2f} 秒")
        print(f"     將丟掉前 {thermal_offset:.2f} 秒的 Thermal 數據")
    
    if time_offset == 0 and thermal_offset == 0:
        print(f"  ✓ 時間對齊完美，無偏移")
    
    print(f"  配對開始時間: {pairing_start_time}")
    
    # 計算新的 RGB 起始幀索引
    rgb_start_frame_idx = int(round(time_offset * fps))
    pairing_end_time = min(rgb_end_time, thermal_end_time)
    duration_seconds = int((pairing_end_time - pairing_start_time).total_seconds())
    
    print(f"  RGB 起始幀索引: {rgb_start_frame_idx}")
    print(f"  配對時長: {duration_seconds} 秒")
    
    # ========================================
    # Step 1: 按秒分組 Thermal 數據
    # ========================================
    thermal_by_second = defaultdict(list)
    
    for thermal_frame in tqdm(thermal_data, desc="按秒分組 Thermal 數據"):
        thermal_ts = thermal_frame['timestamp']
        
        # 只取配對時間範圍內的 Thermal 數據
        if thermal_ts < pairing_start_time or thermal_ts > pairing_end_time:
            continue
        
        # 相對於配對開始時間的偏移
        offset = (thermal_ts - pairing_start_time).total_seconds()
        second_idx = int(offset)
        
        thermal_by_second[second_idx].append({
            'frame': thermal_frame,
            'offset_in_second': offset - second_idx,
            'frame_position': len(thermal_by_second[second_idx])
        })
    
    print(f"\n✅ Thermal 分組完成")
    print(f"  覆蓋秒數: {len(thermal_by_second)}")
    if thermal_by_second:
        print(f"  秒數範圍: {min(thermal_by_second.keys())} ~ {max(thermal_by_second.keys())}")
    
    # ========================================
    # Step 2: 分析每秒的 Thermal 幀數
    # ========================================
    frame_counts_by_second = {sec: len(frames) for sec, frames in thermal_by_second.items()}
    
    print(f"\n📈 Thermal 幀數分布:")
    frame_count_dist = Counter(frame_counts_by_second.values())
    for count in sorted(frame_count_dist.keys()):
        num_seconds = frame_count_dist[count]
        percentage = num_seconds / len(thermal_by_second) * 100 if thermal_by_second else 0
        print(f"  {count:2d} 幀/秒: {num_seconds:4d} 秒 ({percentage:5.1f}%)")
    
    # ========================================
    # Step 3: 識別缺失秒和溢出秒
    # ========================================
    expected_seconds = set(range(duration_seconds + 1))
    actual_seconds = set(thermal_by_second.keys())
    missing_seconds = sorted(expected_seconds - actual_seconds)
    
    overflow_seconds = {sec: thermal_by_second[sec] for sec in thermal_by_second 
                       if frame_counts_by_second[sec] > FRAMES_PER_SECOND}
    
    deficit_seconds = {sec: thermal_by_second[sec] for sec in thermal_by_second 
                      if frame_counts_by_second[sec] < FRAMES_PER_SECOND}
    
    print(f"\n🔍 秒數分析:")
    print(f"  缺失秒數: {len(missing_seconds)} 秒")
    print(f"  溢出秒數: {len(overflow_seconds)} 秒")
    print(f"  不足秒數: {len(deficit_seconds)} 秒")
    
    if missing_seconds:
        missing_display = missing_seconds[:20] if len(missing_seconds) > 20 else missing_seconds
        print(f"  缺失秒: {missing_display}", end="")
        if len(missing_seconds) > 20:
            print(f" ... 還有 {len(missing_seconds)-20} 秒")
        else:
            print()
    
    if overflow_seconds:
        overflow_display = list(overflow_seconds.keys())[:10]
        print(f"  溢出秒: {overflow_display}")
    
    # ========================================
    # Step 4: 補償邏輯
    # ========================================
    overflow_pool = []
    for sec in sorted(overflow_seconds.keys()):
        overflow_frames = overflow_seconds[sec]
        excess = overflow_frames[FRAMES_PER_SECOND:]
        overflow_pool.extend(excess)
    
    print(f"\n🔄 補償過程:")
    print(f"  溢出幀池: {len(overflow_pool)} 幀")
    
    compensation_used = 0
    compensation_map = {}
    
    for missing_sec in missing_seconds:
        if not overflow_pool:
            break
        
        frames_to_fill = min(FRAMES_PER_SECOND, len(overflow_pool))
        compensation_frames = overflow_pool[:frames_to_fill]
        overflow_pool = overflow_pool[frames_to_fill:]
        
        thermal_by_second[missing_sec] = []
        for idx, frame_data in enumerate(compensation_frames):
            new_offset_in_second = idx * FRAME_INTERVAL
            new_timestamp = pairing_start_time + timedelta(
                seconds=missing_sec + new_offset_in_second
            )
            
            thermal_by_second[missing_sec].append({
                'frame': {**frame_data['frame'], 'timestamp': new_timestamp},
                'offset_in_second': new_offset_in_second,
                'frame_position': idx,
                'is_compensated': True
            })
        
        compensation_used += frames_to_fill
        compensation_map[missing_sec] = frames_to_fill
    
    print(f"  已補償秒數: {len(compensation_map)} 秒")
    print(f"  已使用溢出幀: {compensation_used} 幀")
    print(f"  剩餘溢出幀: {len(overflow_pool)} 幀")
    
    # ========================================
    # Step 5: 生成 RGB 幀索引（每秒 8 幀）
    # ========================================
    rgb_frame_indices_by_second = {}
    
    for second_idx in range(duration_seconds + 1):
        indices = []
        for frame_pos in range(FRAMES_PER_SECOND):
            frame_time = pairing_start_time + timedelta(
                seconds=second_idx + frame_pos * FRAME_INTERVAL
            )
            
            offset = (frame_time - camera_start_time).total_seconds()
            rgb_idx = int(round(offset * fps))
            rgb_idx = max(0, min(rgb_idx, frame_count - 1))
            
            indices.append(rgb_idx)
        
        rgb_frame_indices_by_second[second_idx] = indices
    
    print(f"\n✅ RGB 幀索引生成完成")
    
    # ========================================
    # Step 6: 配對
    # ========================================
    pairs = []
    pair_id_counter = 0
    
    pairing_stats = {
        'total_seconds': 0,
        'paired_frames': 0,
        'skipped_frames': 0,
        'frames_per_second_distribution': Counter(),
        'time_offset_seconds': time_offset,
        'pairing_start_time': pairing_start_time
    }
    
    print(f"\n🔗 開始配對...")
    for second_idx in tqdm(sorted(thermal_by_second.keys()), desc="配對 Thermal-RGB"):
        thermal_frames_in_second = thermal_by_second[second_idx]
        num_thermal = len(thermal_frames_in_second)
        
        num_to_pair = min(num_thermal, FRAMES_PER_SECOND)
        rgb_indices = rgb_frame_indices_by_second[second_idx]
        
        pairing_stats['total_seconds'] += 1
        pairing_stats['frames_per_second_distribution'][num_thermal] += 1
        
        for frame_idx in range(num_to_pair):
            thermal_data_item = thermal_frames_in_second[frame_idx]
            thermal_frame = thermal_data_item['frame']
            rgb_frame_idx = rgb_indices[frame_idx]
            
            pairs.append({
                'pair_id': f'pair_{pair_id_counter:05d}',
                'thermal': thermal_frame,
                'rgb_frame_idx': rgb_frame_idx,
                'timestamp': thermal_frame['timestamp'],
                'frame_number': thermal_frame['frame_number'],
                'rgb_error_ms': 0,
                'second_idx': second_idx,
                'frame_in_second': frame_idx,
                'is_compensated': thermal_data_item.get('is_compensated', False)
            })
            
            pair_id_counter += 1
            pairing_stats['paired_frames'] += 1
        
        skipped = FRAMES_PER_SECOND - num_to_pair
        pairing_stats['skipped_frames'] += skipped
    
    # ========================================
    # Step 7: 統計和報告
    # ========================================
    print(f"\n{'='*70}")
    print(f"配對結果統計")
    print(f"{'='*70}")
    
    print(f"\n📊 配對數量:")
    print(f"  成功配對對數: {len(pairs):,}")
    print(f"  配對幀總數: {pairing_stats['paired_frames']:,}")
    print(f"  跳過幀總數: {pairing_stats['skipped_frames']:,}")
    print(f"  覆蓋秒數: {pairing_stats['total_seconds']} 秒")
    
    print(f"\n⏱️ 時間偏移:")
    print(f"  原始 RGB 起始時間: {camera_start_time}")
    print(f"  Thermal 起始時間: {thermal_start_time}")
    print(f"  配對起始時間: {pairing_start_time}")
    print(f"  時間偏移: {time_offset:.2f} 秒")
    
    print(f"\n📈 每秒 Thermal 幀數分布:")
    for frame_count in sorted(pairing_stats['frames_per_second_distribution'].keys()):
        num_secs = pairing_stats['frames_per_second_distribution'][frame_count]
        print(f"  {frame_count:2d} 幀/秒: {num_secs:4d} 秒")
    
    if pairs:
        rgb_indices = [p['rgb_frame_idx'] for p in pairs]
        intervals = [rgb_indices[i+1] - rgb_indices[i] for i in range(len(rgb_indices) - 1)]
        interval_counts = Counter(intervals)
        
        print(f"\n📊 RGB 幀間隔分布:")
        for interval in sorted(interval_counts.keys())[:10]:
            count = interval_counts[interval]
            percentage = count / len(intervals) * 100 if intervals else 0
            print(f"  間隔 {interval:3d} 幀: {count:5d} 次 ({percentage:5.1f}%)")
    
    print(f"\n✅ 補償詳情:")
    if compensation_map:
        print(f"  補償的缺失秒: {len(compensation_map)}")
        for sec, count in sorted(compensation_map.items())[:10]:
            print(f"    秒 {sec}: 補充 {count} 幀")
        if len(compensation_map) > 10:
            print(f"    ... 還有 {len(compensation_map)-10} 秒")
    else:
        print(f"  未進行補償")
    
    uncompensated_missing = len(missing_seconds) - len(compensation_map)
    print(f"\n⚠️ 未補償的缺失秒: {uncompensated_missing}")
    
    print("="*70)
    
    return pairs, {
        'thermal_by_second': thermal_by_second,
        'rgb_frame_indices_by_second': rgb_frame_indices_by_second,
        'compensation_map': compensation_map,
        'overflow_seconds': overflow_seconds,
        'deficit_seconds': deficit_seconds,
        'missing_seconds': missing_seconds,
        'pairing_stats': pairing_stats,
        'time_offset_seconds': time_offset,
        'pairing_start_time': pairing_start_time,
        'rgb_start_frame_idx': rgb_start_frame_idx
    }

# ========================================
# 執行新版配對
# ========================================
pairs_adaptive, pairing_analysis = second_based_pairing_with_time_offset(
    therm_data, camera_start_time, fps, frame_count,
    target_fps=TARGET_FPS, frame_interval_ms=FRAME_INTERVAL_MS
)

print(f"\n📌 最終配對結果: {len(pairs_adaptive):,} 對")
print(f"📌 時間偏移: {pairing_analysis['time_offset_seconds']:.2f} 秒")
print(f"📌 配對起始時間: {pairing_analysis['pairing_start_time']}")


秒級分組配對（改進版 - 帶時間偏移檢測）

📊 檢測時間偏移：
  RGB 時間範圍: 2020-09-24 13:09:29 ~ 2020-09-24 13:20:29.072963
  Thermal 時間範圍: 2020-09-24 13:09:30 ~ 2020-09-24 13:20:34.625000

🔄 時間對齊:
  ⚠️ RGB 比 Thermal 早 1.00 秒
     將丟掉前 1.00 秒的 RGB 數據
  配對開始時間: 2020-09-24 13:09:30
  RGB 起始幀索引: 25
  配對時長: 659 秒


按秒分組 Thermal 數據: 100%|██████████| 5318/5318 [00:00<00:00, 1089552.01it/s]



✅ Thermal 分組完成
  覆蓋秒數: 660
  秒數範圍: 0 ~ 659

📈 Thermal 幀數分布:
   1 幀/秒:    1 秒 (  0.2%)
   8 幀/秒:  659 秒 ( 99.8%)

🔍 秒數分析:
  缺失秒數: 0 秒
  溢出秒數: 0 秒
  不足秒數: 1 秒

🔄 補償過程:
  溢出幀池: 0 幀
  已補償秒數: 0 秒
  已使用溢出幀: 0 幀
  剩餘溢出幀: 0 幀

✅ RGB 幀索引生成完成

🔗 開始配對...


配對 Thermal-RGB: 100%|██████████| 660/660 [00:00<00:00, 90056.30it/s]


配對結果統計

📊 配對數量:
  成功配對對數: 5,273
  配對幀總數: 5,273
  跳過幀總數: 7
  覆蓋秒數: 660 秒

⏱️ 時間偏移:
  原始 RGB 起始時間: 2020-09-24 13:09:29
  Thermal 起始時間: 2020-09-24 13:09:30
  配對起始時間: 2020-09-24 13:09:30
  時間偏移: 1.00 秒

📈 每秒 Thermal 幀數分布:
   1 幀/秒:    1 秒
   8 幀/秒:  659 秒

📊 RGB 幀間隔分布:
  間隔   3 幀:  4831 次 ( 91.6%)
  間隔   4 幀:   441 次 (  8.4%)

✅ 補償詳情:
  未進行補償

⚠️ 未補償的缺失秒: 0

📌 最終配對結果: 5,273 對
📌 時間偏移: 1.00 秒
📌 配對起始時間: 2020-09-24 13:09:30


---
## Step 7: 生成配對驗證影片

In [121]:
# ========================================
# Step 7: 翻轉檢測與修復
# ========================================

def detect_flip_status(
    pairs: List[Dict],
    avi_file: Path,
    sample_size: int = 10,
    default_flip_thermal: bool = True,
    default_flip_rgb: bool = False
) -> Tuple[bool, bool]:
    """
    檢測 Thermal 和 RGB 是否需要翻轉
    基於秒級配對的對應幀進行檢測
    """
    print(f"\n{'='*70}")
    print(f"翻轉檢測")
    print(f"{'='*70}")
    
    cap = cv2.VideoCapture(str(avi_file))
    
    if not cap.isOpened():
        print(f"❌ 無法開啟影片，使用預設值")
        print(f"  翻轉 Thermal: {default_flip_thermal}")
        print(f"  翻轉 RGB: {default_flip_rgb}")
        return default_flip_thermal, default_flip_rgb
    
    sample_pairs = pairs[::max(1, len(pairs) // sample_size)][:sample_size]
    
    print(f"\n📸 檢查 {len(sample_pairs)} 對樣本...")
    
    flip_thermal_votes = 0
    flip_rgb_votes = 0
    valid_samples = 0
    
    for idx, pair in enumerate(sample_pairs):
        rgb_frame_idx = pair['rgb_frame_idx']
        thermal_image = pair['thermal']['image']
        
        cap.set(cv2.CAP_PROP_POS_FRAMES, rgb_frame_idx)
        ret, rgb_frame = cap.read()
        
        if not ret:
            continue
        
        valid_samples += 1
        rgb_gray = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2GRAY)
        
        # 簡單的翻轉檢測：比較邊角亮度
        thermal_tl = thermal_image[0, 0]
        thermal_br = thermal_image[-1, -1]
        
        rgb_tl = rgb_gray[0, 0]
        rgb_br = rgb_gray[-1, -1]
        
        thermal_diff = abs(int(thermal_tl) - int(thermal_br))
        rgb_diff = abs(int(rgb_tl) - int(rgb_br))
        
        if thermal_diff > 50:
            flip_thermal_votes += 1
        if rgb_diff > 50:
            flip_rgb_votes += 1
    
    cap.release()
    
    if valid_samples == 0:
        print(f"⚠️ 無有效樣本，使用預設值")
        flip_thermal_needed = default_flip_thermal
        flip_rgb_needed = default_flip_rgb
    else:
        flip_thermal_needed = flip_thermal_votes > valid_samples / 2
        flip_rgb_needed = flip_rgb_votes > valid_samples / 2
    
    print(f"\n✅ 檢測結果:")
    print(f"  翻轉 Thermal: {flip_thermal_needed}")
    print(f"  翻轉 RGB: {flip_rgb_needed}")
    
    print("="*70)
    
    return flip_thermal_needed, flip_rgb_needed

# 執行翻轉檢測
detected_flip_thermal, detected_flip_rgb = detect_flip_status(
    pairs_adaptive,
    AVI_FILE,
    sample_size=10,
    default_flip_thermal=FLIP_THERMAL,
    default_flip_rgb=FLIP_RGB
)

# ========================================
# Step 7.5: 生成配對影片（新增到此位置）
# ========================================

def generate_pairing_video(
    pairs: List[Dict],
    avi_file: Path,
    output_dir: Path,
    flip_thermal: bool = False,
    flip_rgb: bool = False,
    fps: int = 8,
    video_quality: int = cv2.VideoWriter_fourcc(*'mp4v')
) -> None:
    """
    根據秒級配對結果生成視訊
    將配對的 Thermal 和 RGB 並排顯示
    
    Thermal 特性：
    - 插值放大到與 RGB 相同解析度（INTER_CUBIC）
    - 應用 JET 顏色映射
    - 應用翻轉（與配對邏輯一致）
    """
    print(f"\n{'='*70}")
    print(f"生成配對影片")
    print(f"{'='*70}")
    
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # 打開 RGB 視訊
    cap_rgb = cv2.VideoCapture(str(avi_file))
    if not cap_rgb.isOpened():
        raise RuntimeError(f"❌ 無法開啟 RGB 影片 {avi_file}")
    
    # 獲取 RGB 視訊資訊
    rgb_width = int(cap_rgb.get(cv2.CAP_PROP_FRAME_WIDTH))
    rgb_height = int(cap_rgb.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # 設定輸出視訊參數
    output_width = rgb_width * 2 + 32  # RGB + Thermal + 間隔
    output_height = rgb_height
    
    output_path = output_dir / 'pairing_result.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (output_width, output_height))
    
    if not out.isOpened():
        raise RuntimeError(f"❌ 無法建立視訊寫入器")
    
    print(f"\n📹 視訊參數:")
    print(f"  RGB 解析度: {rgb_width} × {rgb_height}")
    print(f"  輸出解析度: {output_width} × {output_height}")
    print(f"  FPS: {fps}")
    print(f"  翻轉 Thermal: {flip_thermal}")
    print(f"  翻轉 RGB: {flip_rgb}")
    print(f"  Thermal 映射: JET (INTER_CUBIC 插值)")
    print(f"  輸出路徑: {output_path}")
    print(f"  配對總數: {len(pairs)}")
    
    # 快取 RGB 幀
    frame_cache = {}
    
    print(f"\n🎬 生成影片幀...")
    
    for idx, pair in enumerate(tqdm(pairs, desc="生成配對影片")):
        try:
            rgb_frame_idx = pair['rgb_frame_idx']
            thermal_frame = pair['thermal']['image'].copy()
            pair_id = pair['pair_id']
            second_idx = pair['second_idx']
            frame_in_second = pair['frame_in_second']
            
            # 應用 Thermal 翻轉
            if flip_thermal:
                thermal_frame = cv2.flip(thermal_frame, -1)
            
            # 讀取 RGB 幀
            if rgb_frame_idx not in frame_cache:
                cap_rgb.set(cv2.CAP_PROP_POS_FRAMES, rgb_frame_idx)
                ret, rgb_frame = cap_rgb.read()
                if not ret:
                    continue
                
                # 應用 RGB 翻轉
                if flip_rgb:
                    rgb_frame = cv2.flip(rgb_frame, -1)
                
                frame_cache[rgb_frame_idx] = rgb_frame
            else:
                rgb_frame = frame_cache[rgb_frame_idx]
            
            # 準備 Thermal 圖像（可視化）
            thermal_vis = thermal_frame.astype(np.float32)
            thermal_max = thermal_vis.max()
            if thermal_max > 0:
                thermal_vis = (thermal_vis / thermal_max * 255).astype(np.uint8)
            else:
                thermal_vis = thermal_vis.astype(np.uint8)
            
            # 插值放大 Thermal 到與 RGB 相同解析度
            thermal_upscaled = cv2.resize(thermal_vis, (rgb_width, rgb_height),
                                         interpolation=cv2.INTER_CUBIC)
            
            # 應用 JET 顏色映射
            thermal_colored = cv2.applyColorMap(thermal_upscaled, cv2.COLORMAP_JET)
            
            # 創建輸出幀（RGB | Thermal）
            output_frame = np.hstack([rgb_frame, thermal_colored])
            
            # 添加分隔線
            cv2.line(output_frame, (rgb_width + 16, 0), (rgb_width + 16, rgb_height),
                    (255, 255, 255), 2)
            
            # 添加文字信息
            text_info = f"{pair_id} | Sec:{second_idx} Frame:{frame_in_second}"
            cv2.putText(output_frame, text_info, (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # 寫入視訊
            out.write(output_frame)
            
        except Exception as e:
            print(f"⚠️ 處理 {pair_id} 時出錯: {e}")
            continue
    
    # 釋放資源
    cap_rgb.release()
    out.release()
    frame_cache.clear()
    
    print(f"\n✅ 影片生成完成")
    print(f"  輸出路徑: {output_path}")
    if output_path.exists():
        print(f"  檔案大小: {output_path.stat().st_size / (1024*1024):.2f} MB")
    print("="*70)


# 執行生成影片
generate_pairing_video(
    pairs_adaptive,
    AVI_FILE,
    OUTPUT_DIR / 'pairing_videos',
    flip_thermal=detected_flip_thermal,
    flip_rgb=detected_flip_rgb,
    fps=TARGET_FPS
)


翻轉檢測

📸 檢查 10 對樣本...

✅ 檢測結果:
  翻轉 Thermal: False
  翻轉 RGB: False

生成配對影片

📹 視訊參數:
  RGB 解析度: 640 × 360
  輸出解析度: 1312 × 360
  FPS: 8
  翻轉 Thermal: False
  翻轉 RGB: False
  Thermal 映射: JET (INTER_CUBIC 插值)
  輸出路徑: /home/kirito/USED/output_v6/pairing_videos/pairing_result.mp4
  配對總數: 5273

🎬 生成影片幀...


生成配對影片: 100%|██████████| 5273/5273 [00:48<00:00, 109.72it/s]



✅ 影片生成完成
  輸出路徑: /home/kirito/USED/output_v6/pairing_videos/pairing_result.mp4
  檔案大小: 0.00 MB


---
## Step 8 - 匯出 RGB 圖片供 LabelMe 標註

In [122]:
# ========================================
# Step 8: 匯出 RGB 圖片供 LabelMe 標註
# ========================================

def export_rgb_images_for_labelme(
    pairs: List[Dict],
    avi_file: Path,
    output_dir: Path,
    flip_rgb: bool = False,
    sample_interval: int = 1
) -> pd.DataFrame:
    """
    根據秒級配對結果，匯出 RGB 圖片
    每秒只匯出 1 張（或更少，取決於 Thermal 幀數）
    """
    print(f"\n{'='*70}")
    print(f"匯出 RGB 圖片供 LabelMe 標註")
    print(f"{'='*70}")
    
    output_dir.mkdir(parents=True, exist_ok=True)
    cap = cv2.VideoCapture(str(avi_file))
    
    if not cap.isOpened():
        raise RuntimeError(f"❌ 無法開啟影片 {avi_file}")
    
    # 為了減少重複讀取，快取 RGB 幀
    frame_cache = {}
    
    metadata = []
    
    print(f"\n📸 匯出圖片...")
    
    # 按秒級分組匯出（每秒只匯出該秒第一張配對）
    exported_count = 0
    
    for pair in tqdm(pairs, desc="匯出 RGB 圖片"):
        if pair['frame_in_second'] % sample_interval != 0:
            continue  # 只匯出該秒的第一張
        
        rgb_frame_idx = pair['rgb_frame_idx']
        pair_id = pair['pair_id']
        
        if rgb_frame_idx not in frame_cache:
            cap.set(cv2.CAP_PROP_POS_FRAMES, rgb_frame_idx)
            ret, frame = cap.read()
            if not ret:
                continue
            
            if flip_rgb:
                frame = cv2.flip(frame, -1)  # 水平和垂直翻轉
            
            frame_cache[rgb_frame_idx] = frame
        else:
            frame = frame_cache[rgb_frame_idx]
        
        # 儲存圖片
        save_path = output_dir / f"frame_{pair_id}.jpg"
        cv2.imwrite(str(save_path), frame)
        
        metadata.append({
            'pair_id': pair_id,
            'filename': f"frame_{pair_id}.jpg",
            'rgb_frame_idx': rgb_frame_idx,
            'timestamp': str(pair['timestamp']),
            'second_idx': pair['second_idx'],
            'frame_in_second': pair['frame_in_second'],
            'is_compensated': pair['is_compensated']
        })
        
        exported_count += 1
    
    cap.release()
    
    # 清空快取
    frame_cache.clear()
    
    print(f"\n✅ 匯出完成")
    print(f"  圖片數: {exported_count}")
    print(f"  儲存位置: {output_dir}")
    
    # 儲存 metadata
    metadata_df = pd.DataFrame(metadata)
    metadata_csv = output_dir.parent / 'image_metadata.csv'
    metadata_df.to_csv(metadata_csv, index=False, encoding='utf-8-sig')
    
    print(f"  Metadata: {metadata_csv}")
    print("="*70)
    
    return metadata_df

# 執行匯出
rgb_metadata = export_rgb_images_for_labelme(
    pairs_adaptive,
    AVI_FILE,
    LABELME_DIR / 'images',
    flip_rgb=detected_flip_rgb,
    sample_interval=1
)


匯出 RGB 圖片供 LabelMe 標註

📸 匯出圖片...


匯出 RGB 圖片: 100%|██████████| 5273/5273 [00:50<00:00, 104.51it/s]



✅ 匯出完成
  圖片數: 5273
  儲存位置: /home/kirito/USED/output_v6/labelme_project/images
  Metadata: /home/kirito/USED/output_v6/labelme_project/image_metadata.csv


---
## Step 9: 儲存配對結果

In [123]:
# ========================================
# Step 9: 儲存對齊後的 Thermal 資料
# ========================================

def extract_and_save_thermal_data(
    pairs: List[Dict],
    output_dir: Path,
    flip_thermal: bool = False
) -> pd.DataFrame:
    """
    根據秒級配對結果，儲存對齊後的 Thermal 數據
    """
    print(f"\n{'='*70}")
    print(f"儲存對齊後的 Thermal 數據")
    print(f"{'='*70}")
    
    thermal_dir = output_dir / 'thermal'
    thermal_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"\n💾 儲存 Thermal 圖片...")
    
    metadata = []
    
    for pair in tqdm(pairs, desc="儲存 Thermal 數據"):
        pair_id = pair['pair_id']
        thermal_image = pair['thermal']['image'].copy()
        
        # 應用翻轉
        if flip_thermal:
            thermal_image = cv2.flip(thermal_image, -1)
        
        # 儲存為 .npy（保留原始數值用於訓練）
        npy_path = thermal_dir / f"{pair_id}_thermal.npy"
        np.save(npy_path, thermal_image)
        
        # 也儲存為 PNG 用於可視化
        png_path = thermal_dir / f"{pair_id}_thermal.png"
        thermal_max = thermal_image.max()
        if thermal_max > 0:
            thermal_vis = (thermal_image / thermal_max * 255).astype(np.uint8)
        else:
            thermal_vis = thermal_image.astype(np.uint8)
        thermal_vis = cv2.applyColorMap(thermal_vis, cv2.COLORMAP_JET)
        cv2.imwrite(str(png_path), thermal_vis)
        
        metadata.append({
            'pair_id': pair_id,
            'thermal_file': f"{pair_id}_thermal.npy",
            'thermal_png': f"{pair_id}_thermal.png",
            'frame_number': pair['frame_number'],
            'timestamp': str(pair['timestamp']),
            'rgb_frame_idx': pair['rgb_frame_idx'],
            'second_idx': pair['second_idx'],
            'frame_in_second': pair['frame_in_second'],
            'thermal_shape': str(thermal_image.shape),
            'is_compensated': pair['is_compensated']
        })
    
    print(f"\n✅ 儲存完成")
    print(f"  Thermal 數據: {len(metadata)} 組")
    print(f"  儲存位置: {thermal_dir}")
    
    # 儲存 metadata
    metadata_df = pd.DataFrame(metadata)
    metadata_csv = output_dir / 'metadata.csv'
    metadata_df.to_csv(metadata_csv, index=False, encoding='utf-8-sig')
    
    print(f"  Metadata: {metadata_csv}")
    print("="*70)
    
    return metadata_df

# 執行儲存
thermal_metadata = extract_and_save_thermal_data(
    pairs_adaptive,
    ALIGNED_DIR,
    flip_thermal=detected_flip_thermal
)


儲存對齊後的 Thermal 數據

💾 儲存 Thermal 圖片...


儲存 Thermal 數據: 100%|██████████| 5273/5273 [00:02<00:00, 1816.58it/s]



✅ 儲存完成
  Thermal 數據: 5273 組
  儲存位置: /home/kirito/USED/output_v6/aligned_dataset/thermal
  Metadata: /home/kirito/USED/output_v6/aligned_dataset/metadata.csv


---
## Step 10: 驗證修正結果

In [124]:
# ========================================
# Step 10: 儲存配對映射 JSON
# ========================================

def save_pairing_mapping(
    pairs: List[Dict],
    pairing_analysis: Dict,
    output_dir: Path
) -> None:
    """
    儲存詳細的配對映射信息（用於後續訓練）
    """
    print(f"\n{'='*70}")
    print(f"儲存配對映射信息")
    print(f"{'='*70}")
    
    # 準備序列化友好的格式
    pairs_serializable = []
    for pair in pairs:
        pairs_serializable.append({
            'pair_id': pair['pair_id'],
            'thermal_file': f"{pair['pair_id']}_thermal.npy",
            'rgb_frame_idx': pair['rgb_frame_idx'],
            'timestamp': str(pair['timestamp']),
            'frame_number': pair['frame_number'],
            'second_idx': pair['second_idx'],
            'frame_in_second': pair['frame_in_second'],
            'is_compensated': pair['is_compensated']
        })
    
    pairing_data = {
        'total_pairs': len(pairs),
        'pairs': pairs_serializable,
        'time_offset_info': {
            'time_offset_seconds': pairing_analysis['time_offset_seconds'],
            'pairing_start_time': str(pairing_analysis['pairing_start_time']),
            'rgb_start_frame_idx': pairing_analysis['rgb_start_frame_idx']
        },
        'statistics': {
            'total_seconds': pairing_analysis['pairing_stats']['total_seconds'],
            'paired_frames': pairing_analysis['pairing_stats']['paired_frames'],
            'skipped_frames': pairing_analysis['pairing_stats']['skipped_frames'],
            'frames_per_second_distribution': {
                str(k): v for k, v in pairing_analysis['pairing_stats']['frames_per_second_distribution'].items()
            },
            'compensation': {
                'total_compensated_seconds': len(pairing_analysis['compensation_map']),
                'missing_seconds_count': len(pairing_analysis['missing_seconds']),
                'overflow_seconds_count': len(pairing_analysis['overflow_seconds'])
            }
        }
    }
    
    output_dir.mkdir(parents=True, exist_ok=True)
    output_json = output_dir / 'pairs_mapping.json'
    
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(pairing_data, f, indent=2, ensure_ascii=False, default=str)
    
    print(f"\n✅ 配對映射已儲存: {output_json}")
    print(f"  總對數: {len(pairs)}")
    print(f"  補償秒數: {len(pairing_analysis['compensation_map'])}")
    print(f"  缺失秒數: {len(pairing_analysis['missing_seconds'])}")
    print(f"  時間偏移: {pairing_analysis['time_offset_seconds']:.2f} 秒")
    print("="*70)

# 執行儲存
save_pairing_mapping(pairs_adaptive, pairing_analysis, PAIRING_DIR)


儲存配對映射信息

✅ 配對映射已儲存: /home/kirito/USED/output_v6/pairing_analysis/pairs_mapping.json
  總對數: 5273
  補償秒數: 0
  缺失秒數: 0
  時間偏移: 1.00 秒


---
##  Step 11 - 生成配對分析可視化

In [125]:
# ========================================
# Step 11: 生成配對分析可視化
# ========================================

def visualize_pairing_analysis(pairing_analysis: Dict, output_dir: Path) -> None:
    """
    視覺化配對分析結果
    """
    print(f"\n{'='*70}")
    print(f"生成配對分析可視化")
    print(f"{'='*70}")
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. 每秒 Thermal 幀數分布
    dist = pairing_analysis['pairing_stats']['frames_per_second_distribution']
    frames_count = sorted(dist.keys())
    frames_freq = [dist[f] for f in frames_count]
    
    axes[0, 0].bar(range(len(frames_count)), frames_freq, color='steelblue', edgecolor='black')
    axes[0, 0].set_xticks(range(len(frames_count)))
    axes[0, 0].set_xticklabels(frames_count)
    axes[0, 0].set_xlabel('Frames per Second')
    axes[0, 0].set_ylabel('Count')
    axes[0, 0].set_title('Thermal Frames per Second Distribution')
    axes[0, 0].grid(axis='y', alpha=0.3)
    
    # 2. 補償情況圓餅圖
    compensation_map = pairing_analysis['compensation_map']
    missing_count = len(pairing_analysis['missing_seconds'])
    compensated_count = len(compensation_map)
    uncompensated_count = missing_count - compensated_count
    
    if missing_count > 0:
        sizes = [compensated_count, uncompensated_count]
        labels = [f'Compensated\n({compensated_count})', f'Missing\n({uncompensated_count})']
        colors = ['#2ecc71', '#e74c3c']
        axes[0, 1].pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    else:
        axes[0, 1].text(0.5, 0.5, 'No Missing Seconds', ha='center', va='center', fontsize=14)
    
    axes[0, 1].set_title('Missing Seconds Compensation Status')
    
    # 3. 秒別配對幀數趨勢圖
    thermal_by_second = pairing_analysis['thermal_by_second']
    second_indices = sorted(thermal_by_second.keys())
    frame_counts_per_sec = [len(thermal_by_second[s]) for s in second_indices]
    
    # 按秒數分段展示（前 100 秒）
    segment_size = min(100, len(second_indices))
    segment_seconds = second_indices[:segment_size]
    segment_counts = frame_counts_per_sec[:segment_size]
    
    axes[1, 0].plot(range(len(segment_counts)), segment_counts, marker='o', linestyle='-', linewidth=1, markersize=3, color='steelblue')
    axes[1, 0].set_xlabel('Second Index')
    axes[1, 0].set_ylabel('Thermal Frames in Second')
    axes[1, 0].set_title(f'Thermal Frames per Second Trend (First {segment_size}s)')
    axes[1, 0].grid(alpha=0.3)
    axes[1, 0].axhline(y=8, color='r', linestyle='--', label='Target (8 frames)')
    axes[1, 0].legend()
    
    # 4. 統計摘要表
    stats_text = f"""
配對統計摘要

總配對對數: {pairing_analysis['pairing_stats']['total_seconds']}
覆蓋秒數: {pairing_analysis['pairing_stats']['total_seconds']}
配對幀總數: {pairing_analysis['pairing_stats']['paired_frames']}
跳過幀總數: {pairing_analysis['pairing_stats']['skipped_frames']}

缺失秒數: {len(pairing_analysis['missing_seconds'])}
溢出秒數: {len(pairing_analysis['overflow_seconds'])}
不足秒數: {len(pairing_analysis['deficit_seconds'])}

已補償秒數: {len(pairing_analysis['compensation_map'])}
未補償秒數: {len(pairing_analysis['missing_seconds']) - len(pairing_analysis['compensation_map'])}

時間偏移: {pairing_analysis['time_offset_seconds']:.2f} 秒
    """
    
    axes[1, 1].text(0.1, 0.5, stats_text, fontsize=11, family='monospace',
                    verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    output_dir.mkdir(parents=True, exist_ok=True)
    save_path = output_dir / 'pairing_analysis_v2.png'
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"\n✅ 可視化已儲存: {save_path}")
    print("="*70)

# 執行可視化
visualize_pairing_analysis(pairing_analysis, PAIRING_DIR)


生成配對分析可視化

✅ 可視化已儲存: /home/kirito/USED/output_v6/pairing_analysis/pairing_analysis_v2.png
